In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/pipelines/google_cloud_pipeline_components_bqml_pipeline_demand_forecasting.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/pipelines/google_cloud_pipeline_components_bqml_pipeline_demand_forecasting.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/pipelines/google_cloud_pipeline_components_bqml_pipeline_demand_forecasting.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>                                                                                               
</table>

## Overview

This notebook shows how to use `Vertex AI Pipelines` and `BigQuery ML pipeline components` to train and evaluate a demand forecasting model.

### Dataset

The dataset is a modified version of the dataset in [Build and visualize demand forecast predictions using Datastream, Dataflow, BigQuery ML, and Looker
](https://cloud.google.com/architecture/build-visualize-demand-forecast-prediction-datastream-dataflow-bigqueryml-looker) solution architecture

### Objective

In this tutorial, you learn how to train and evaluate a BigQuery ML model using Vertex AI Pipelines and BigQuery ML pipeline components. 

This tutorial uses the following Google Cloud ML services and resources:

- `Vertex AI Pipelines`
- `BigQuery ML pipeline components`

The steps performed include:

- Define a custom evaluation component
- Define a pipeline:
  - Get BigQuery training data
  - Train a BigQuery Arima Plus model
  - Evaluate the BigQuery Arima Plus model
  - Plot the evaluations
  - Check the model performance
  - Generate the ARIMA Plus forecasts
  - Generate the ARIMA PLUS forecast explainations
- Compile the pipeline.
- Execute the pipeline.

### Costs 

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage
* BigQuery

### Set up your local development environment

**If you are using Colab or Vertex AI Workbench Notebooks**, your environment already meets
all the requirements to run this notebook. You can skip this step.

**Otherwise**, make sure your environment meets this notebook's requirements.
You need the following:

* The Google Cloud SDK
* Git
* Python 3
* virtualenv
* Jupyter notebook running in a virtual environment with Python 3

The Google Cloud guide to [Setting up a Python development
environment](https://cloud.google.com/python/setup) and the [Jupyter
installation guide](https://jupyter.org/install) provide detailed instructions
for meeting these requirements. The following steps provide a condensed set of
instructions:

1. [Install and initialize the Cloud SDK.](https://cloud.google.com/sdk/docs/)

1. [Install Python 3.](https://cloud.google.com/python/setup#installing_python)

1. [Install
   virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)
   and create a virtual environment that uses Python 3. Activate the virtual environment.

1. To install Jupyter, run `pip3 install jupyter` on the
command-line in a terminal shell.

1. To launch Jupyter, run `jupyter notebook` on the command-line in a terminal shell.

1. Open this notebook in the Jupyter Notebook Dashboard.

### Install additional packages

Install the following packages required to execute this notebook.

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"
    
! pip3 install --upgrade "kfp" \
                         "google-cloud-aiplatform" \
                         "google_cloud_pipeline_components" \
                         "google-cloud-bigquery" {USER_FLAG} -q

! pip3 install --upgrade "tensorflow<2.8.0" {USER_FLAG} -q

### Restart the kernel

After you install the additional packages, you need to restart the notebook kernel so it can find the packages.

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Before you begin

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

1. [Enable the API](https://console.cloud.google.com/flows/enableapi?apiid=cloudresourcemanager.googleapis.com,aiplatform.googleapis.com,notebooks.googleapis.com).

1. If you are running this notebook locally, you will need to install the [Cloud SDK](https://cloud.google.com/sdk).

1. Enter your project ID in the cell below. Then run the cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [ ]:
import os

PROJECT_ID = ""

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

Otherwise, set your project ID here.

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = "[your-project]"  # @param {type:"string"}

In [ ]:
! gcloud config set project $PROJECT_ID

#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex AI. We recommend that you choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You may not use a multi-regional bucket for training with Vertex AI. Not all regions provide support for all Vertex AI services.

Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append it onto the name of resources you create in this tutorial.

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### Authenticate your Google Cloud account

**If you are using Vertex AI Workbench Notebooks**, your environment is already
authenticated. Skip this step.

**If you are using Colab**, run the cell below and follow the instructions
when prompted to authenticate your account via oAuth.

**Otherwise**, follow these steps:

1. In the Cloud Console, go to the [**Create service account key**
   page](https://console.cloud.google.com/apis/credentials/serviceaccountkey).

2. Click **Create service account**.

3. In the **Service account name** field, enter a name, and
   click **Create**.

4. In the **Grant this service account access to project** section, click the **Role** drop-down list. Type and select the following roles into the filter box:

  - BigQuery Data Editor
  - BigQuery Job User
  - Service Account User
  - Storage Object Admin
  - Storgae Admin
  - Vertex AI Administrator

5. Click *Create*. A JSON file that contains your key downloads to your
local environment.

6. Enter the path to your service account key as the
`GOOGLE_APPLICATION_CREDENTIALS` variable in the cell below and run the cell.

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

Set the name of your Cloud Storage bucket below. It must be unique across all
Cloud Storage buckets.

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "-aip-" + TIMESTAMP
    BUCKET_URI = f"gs://{BUCKET_NAME}"

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

Finally, validate access to your Cloud Storage bucket by examining its contents:

In [ ]:
! gsutil ls -al $BUCKET_URI

### Get your service account 

If you do not want to use your project's Compute Engine service account, set `SERVICE_ACCOUNT` to another service account ID.

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [ ]:
if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    else:  # IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

### Set service account access 

Run the following commands to grant your service account access to the bucket that you created in the previous step. You only need to run this step once per service account.

In [ ]:
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator {BUCKET_URI}

! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer {BUCKET_URI}

### Import libraries and Set up variables

Next, import libraries and set up some variables used throughout the tutorial.


In [ ]:
from pathlib import Path as path
from typing import NamedTuple
# General
from urllib.parse import urlparse

import google.cloud.aiplatform as vertex_ai
# Check components
import tensorflow as tf
# Simulate operations
from google.cloud import bigquery
# ML pipeline
from google_cloud_pipeline_components.v1.bigquery import (
    BigqueryCreateModelJobOp, BigqueryEvaluateModelJobOp,
    BigqueryExplainForecastModelJobOp, BigqueryForecastModelJobOp,
    BigqueryMLArimaEvaluateJobOp, BigqueryQueryJobOp)
from kfp.v2 import compiler, dsl
from kfp.v2.dsl import HTML, Artifact, Condition, Input, Output, component

### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [ ]:
vertex_ai.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

### Initialize BigQuery SDK for Python

Initialize the BigQuery SDK for Python for your project.

In [ ]:
bq_client = bigquery.Client(project=PROJECT_ID, location=REGION)

### Create local directories

Next, you create some local directories that you use in this tutorial.

In [ ]:
DATA_PATH = "data"
KFP_COMPONENTS_PATH = "components"
PIPELINES_PATH = "pipelines"

! mkdir -m 777 -p {KFP_COMPONENTS_PATH}
! mkdir -m 777 -p {PIPELINES_PATH}

### Prepare the training data

Next, you will make a copy of the CSV training data into your Cloud Storage bucket and then create a BigQuery dataset table for the training data.

In [ ]:
PUBLIC_DATA_URI = "gs://cloud-samples-data/vertex-ai/pipeline-deployment/datasets/oracle_retail/orders.csv"
RAW_DATA_URI = f"{BUCKET_URI}/{DATA_PATH}/raw/orders.csv"

! gsutil -m cp -R $PUBLIC_DATA_URI $RAW_DATA_URI

#### Quick peek at the CSV data

In [ ]:
! gsutil cat {RAW_DATA_URI} | head

In [ ]:
LOCATION = REGION.split('-')[0]

! bq mk --location={LOCATION} --dataset {PROJECT_ID}:fast_fresh 

! bq load \
  --location={LOCATION} \
  --source_format=CSV \
  --skip_leading_rows=1\
  fast_fresh.orders_{TIMESTAMP} \
  {RAW_DATA_URI} \
  time_of_sale:DATETIME,order_id:INTEGER,product_name:STRING,price:NUMERIC,quantity:NUMERIC,payment_method:STRING,store_id:INTEGER,user_id:INTEGER

## BQML Training Formalization

In the next cells, you build the components and pipeline to train and evaluate the BQML demand forecasting model.

### Set up pipeline variables

Set up some variables specific for the pipeline.


In [ ]:
# BQML pipeline job configuation
PIPELINE_NAME = "bqml-forecast-pipeline"
PIPELINE_ROOT = urlparse(BUCKET_URI)._replace(path="pipeline_root").geturl()
PIPELINE_PACKAGE = str(path(PIPELINES_PATH) / f"{PIPELINE_NAME}.json")

# BQML pipeline conponent configuration
BQ_DATASET = "fast_fresh"
BQ_ORDERS_TABLE_PREFIX = "orders"
BQ_TRAINING_TABLE_PREFIX = "orders_training"
BQ_MODEL_TABLE_PREFIX = "orders_forecast_arima"
BQ_EVALUATE_TS_TABLE_PREFIX = "orders_arima_time_series_evaluate"
BQ_EVALUATE_MODEL_TABLE_PREFIX = "orders_arima_model_evaluate"
BQ_FORECAST_TABLE_PREFIX = "orders_arima_forecast"
BQ_EXPLAIN_FORECAST_TABLE_PREFIX = "orders_arima_explain_forecast"
BQ_ORDERS_TABLE = f"{BQ_ORDERS_TABLE_PREFIX}_{TIMESTAMP}"
BQ_TRAINING_TABLE = f"{BQ_TRAINING_TABLE_PREFIX}_{TIMESTAMP}"
BQ_MODEL_TABLE = f"{BQ_MODEL_TABLE_PREFIX}_{TIMESTAMP}"
BQ_EVALUATE_TS_TABLE = f"{BQ_EVALUATE_TS_TABLE_PREFIX}_{TIMESTAMP}"
BQ_EVALUATE_MODEL_TABLE = f"{BQ_EVALUATE_MODEL_TABLE_PREFIX}_{TIMESTAMP}"
BQ_FORECAST_TABLE = f"{BQ_FORECAST_TABLE_PREFIX}_{TIMESTAMP}"
BQ_EXPLAIN_FORECAST_TABLE = f"{BQ_EXPLAIN_FORECAST_TABLE_PREFIX}_{TIMESTAMP}"

BQ_TRAIN_CONFIGURATION = {
    "destinationTable": {
        "projectId": PROJECT_ID,
        "datasetId": BQ_DATASET,
        "tableId": BQ_TRAINING_TABLE,
    },
    "writeDisposition": "WRITE_TRUNCATE",
}

BQ_EVALUATE_TS_CONFIGURATION = {
    "destinationTable": {
        "projectId": PROJECT_ID,
        "datasetId": BQ_DATASET,
        "tableId": BQ_EVALUATE_TS_TABLE,
    },
    "writeDisposition": "WRITE_TRUNCATE",
}
BQ_EVALUATE_MODEL_CONFIGURATION = {
    "destinationTable": {
        "projectId": PROJECT_ID,
        "datasetId": BQ_DATASET,
        "tableId": BQ_EVALUATE_MODEL_TABLE,
    },
    "writeDisposition": "WRITE_TRUNCATE",
}
BQ_FORECAST_CONFIGURATION = {
    "destinationTable": {
        "projectId": PROJECT_ID,
        "datasetId": BQ_DATASET,
        "tableId": BQ_FORECAST_TABLE,
    },
    "writeDisposition": "WRITE_TRUNCATE",
}
BQ_EXPLAIN_FORECAST_CONFIGURATION = {
    "destinationTable": {
        "projectId": PROJECT_ID,
        "datasetId": BQ_DATASET,
        "tableId": BQ_EXPLAIN_FORECAST_TABLE,
    },
    "writeDisposition": "WRITE_TRUNCATE",
}
PERF_THRESHOLD = 3000

### Create location for storing compoment definitions

Next, you create a Storage Cloud location for storing the YAML component definitions for the custom components you create in this tutorial.

In [ ]:
! mkdir -m 777 -p {KFP_COMPONENTS_PATH}/custom_components

### Create a custom component to read model evaluation metrics

Build a custom component to consume model evaluation metrics for visualizations in the Vertex AI Pipelines UI using Kubeflow SDK visualization APIs. Indeed, Vertex AI allows you to render that HTML in an output page which is easily accessible from the Google Cloud console.

In [ ]:
@component(
    base_image="python:3.8-slim",
    packages_to_install=["jinja2", "pandas", "matplotlib"],
    output_component_file=f"{KFP_COMPONENTS_PATH}/custom_components/build_bq_evaluate_metrics.yaml",
)
def get_model_evaluation_metrics(
    metrics_in: Input[Artifact], metrics_out: Output[HTML]
) -> NamedTuple("Outputs", [("avg_mean_absolute_error", float)]):
    """
    Get the average mean absolute error from the metrics
    Args:
        metrics_in: metrics artifact
        metrics_out: metrics artifact
    Returns:
        avg_mean_absolute_error: average mean absolute error
    """

    import pandas as pd

    # Helpers
    def prettyfier(styler):
        """
        Helper function to prettify the metrics table.
        Args:
            styler: Styler object
        Returns:
            Styler object
        """
        caption = {
            "selector": "caption",
            "props": [
                ("caption-side", "top"),
                ("font-size", "150%"),
                ("font-weight", "bold"),
                ("font-family", "arial"),
            ],
        }
        headers = {
            "selector": "th",
            "props": [("color", "black"), ("font-family", "arial")],
        }
        rows = {
            "selector": "td",
            "props": [("text-align", "center"), ("font-family", "arial")],
        }
        styler.set_table_styles([caption, headers, rows])
        styler.set_caption("Forecasting accuracy report <br><br>")
        styler.hide(axis="index")
        styler.format(precision=2)
        styler.background_gradient(cmap="Blues")
        return styler

    def get_column_names(header):
        """
        Helper function to get the column names from the metrics table.
        Args:
            header: header
        Returns:
            column_names: column names
        """
        header_clean = header.replace("_", " ")
        header_abbrev = "".join([h[0].upper() for h in header_clean.split()])
        header_prettied = f"{header_clean} ({header_abbrev})"
        return header_prettied

    # Extract rows and schema from metrics artifact
    rows = metrics_in.metadata["rows"]
    schema = metrics_in.metadata["schema"]

    # Convert into a tabular format
    columns = [metrics["name"] for metrics in schema["fields"] if "name" in metrics]
    records = []
    for row in rows:
        records.append([dl["v"] for dl in row["f"]])
    metrics = (
        pd.DataFrame.from_records(records, columns=columns, index="product_name")
        .astype(float)
        .round(3)
    )
    metrics = metrics.reset_index()

    # Create the HTML artifact for the metrics
    pretty_columns = list(
        map(
            lambda h: get_column_names(h)
            if h != columns[0]
            else h.replace("_", " ").capitalize(),
            columns,
        )
    )
    pretty_metrics = metrics.copy()
    pretty_metrics.columns = pretty_columns
    html_metrics = pretty_metrics.style.pipe(prettyfier).to_html()
    with open(metrics_out.path, "w") as f:
        f.write(html_metrics)

    # Create metrics dictionary for the model
    avg_mean_absolute_error = round(float(metrics.mean_absolute_error.mean()), 0)
    component_outputs = NamedTuple("Outputs", [("avg_mean_absolute_error", float)])

    return component_outputs(avg_mean_absolute_error)

### Build the BQML training pipeline

Define your workflow using Kubeflow Pipelines DSL package. 

Below you have the steps of the pipeline workflow:

1. Get BigQuery training data
2. Train a BigQuery Arima Plus model
3. Evaluate the BigQuery Arima Plus model
4. Plot the evaluations
5. Check the model performance
6. Generate the ARIMA Plus forecasts
7. Generate the ARIMA PLUS forecast explainations

In [ ]:
@dsl.pipeline(
    name=PIPELINE_NAME,
    description="A batch pipeline to train ARIMA PLUS using BQML",
)
def pipeline(
    bq_dataset: str = BQ_DATASET,
    bq_orders_table: str = BQ_ORDERS_TABLE,
    bq_training_table: str = BQ_TRAINING_TABLE,
    bq_train_configuration: dict = BQ_TRAIN_CONFIGURATION,
    bq_model_table: str = BQ_MODEL_TABLE,
    bq_evaluate_time_series_configuration: dict = BQ_EVALUATE_TS_CONFIGURATION,
    bq_evaluate_model_configuration: dict = BQ_EVALUATE_MODEL_CONFIGURATION,
    performance_threshold: float = PERF_THRESHOLD,
    bq_forecast_configuration: dict = BQ_FORECAST_CONFIGURATION,
    bq_explain_forecast_configuration: dict = BQ_EXPLAIN_FORECAST_CONFIGURATION,
    project: str = PROJECT_ID,
    location: str = LOCATION,
):

    # Create the training dataset
    create_training_dataset_op = BigqueryQueryJobOp(
        query=f"""
        -- create the training table
        WITH 
        -- get 90% percentile for time series splitting
        get_split AS (
          SELECT APPROX_QUANTILES(DATETIME_TRUNC(time_of_sale, HOUR), 100)[OFFSET(90)] as split
          FROM `{project}.{bq_dataset}.{bq_orders_table}`
        ),
        -- get train table
        get_train AS (
          SELECT
            DATETIME_TRUNC(time_of_sale, HOUR) as hourly_timestamp,
            product_name,
            SUM(quantity) AS total_sold,
            FROM `{project}.{bq_dataset}.{bq_orders_table}`
        GROUP BY hourly_timestamp, product_name
        )
        SELECT
          *,
          CASE WHEN hourly_timestamp < (SELECT split FROM get_split) THEN 'TRAIN' ELSE 'TEST' END AS split
        FROM get_train
        ORDER BY hourly_timestamp
        """,
        job_configuration_query=bq_train_configuration,
        project=project,
        location=location,
    ).set_display_name("get train data")

    # Train the ARIMA PLUS model
    bq_arima_model_op = (
        BigqueryCreateModelJobOp(
            query=f"""
        -- create model table
        CREATE OR REPLACE MODEL `{project}.{bq_dataset}.{bq_model_table}`
        OPTIONS(
        MODEL_TYPE = \'ARIMA_PLUS\',
        TIME_SERIES_TIMESTAMP_COL = \'hourly_timestamp\',
        TIME_SERIES_DATA_COL = \'total_sold\',
        TIME_SERIES_ID_COL = [\'product_name\'],
        MODEL_REGISTRY = \'vertex_ai\',
        VERTEX_AI_MODEL_ID = \'order_demand_forecasting\',
        VERTEX_AI_MODEL_VERSION_ALIASES = [\'staging\']
        ) AS
        SELECT
          hourly_timestamp,
          product_name,
          total_sold
        FROM `{project}.{bq_dataset}.{bq_training_table}`
        WHERE split='TRAIN';
        """,
            project=project,
            location=location,
        )
        .set_display_name("train arima plus model")
        .after(create_training_dataset_op)
    )

    # Evaluate ARIMA PLUS time series
    _ = (
        BigqueryMLArimaEvaluateJobOp(
            project=project,
            location=location,
            model=bq_arima_model_op.outputs["model"],
            show_all_candidate_models=False,
            job_configuration_query=bq_evaluate_time_series_configuration,
        )
        .set_display_name("evaluate arima plus time series")
        .after(bq_arima_model_op)
    )

    # Evaluate ARIMA Plus model
    bq_arima_evaluate_model_op = (
        BigqueryEvaluateModelJobOp(
            project=project,
            location=location,
            model=bq_arima_model_op.outputs["model"],
            query_statement=f"""SELECT * FROM `{project}.{bq_dataset}.{bq_training_table}` WHERE split='TEST'""",
            job_configuration_query=bq_evaluate_model_configuration,
        )
        .set_display_name("evaluate arima plus model")
        .after(bq_arima_model_op)
    )

    # Plot model metrics
    get_evaluation_model_metrics_op = (
        get_model_evaluation_metrics(
            bq_arima_evaluate_model_op.outputs["evaluation_metrics"]
        )
        .after(bq_arima_evaluate_model_op)
        .set_display_name("plot evaluation metrics")
    )

    # Check the model performance. If ARIMA_PLUS average MAE metric is below to a minimal threshold
    with Condition(
        get_evaluation_model_metrics_op.outputs["avg_mean_absolute_error"]
        < PERF_THRESHOLD,
        name="avg. mae good",
    ):

        # Generate the ARIMA PLUS forecasts
        bq_arima_forecast_op = (
            BigqueryForecastModelJobOp(
                project=project,
                location=location,
                model=bq_arima_model_op.outputs["model"],
                horizon=1,  # 1 hour
                confidence_level=0.9,
                job_configuration_query=bq_forecast_configuration,
            )
            .set_display_name("generate hourly forecasts")
            .after(get_evaluation_model_metrics_op)
        )

        # Generate the ARIMA PLUS forecast explainations
        _ = (
            BigqueryExplainForecastModelJobOp(
                project=project,
                location=location,
                model=bq_arima_model_op.outputs["model"],
                horizon=1,  # 1 hour
                confidence_level=0.9,
                job_configuration_query=bq_explain_forecast_configuration,
            )
            .set_display_name("explain hourly forecasts")
            .after(bq_arima_forecast_op)
        )

### Compile the pipeline into a JSON file

Next, you compile the pipeline, which produces a JSON specification for your pipeline.

In [ ]:
compiler.Compiler().compile(pipeline_func=pipeline, package_path=PIPELINE_PACKAGE)

### Execute your pipeline

Next, you execute the pipeline. It takes the following parameters which we set as default:

- `bq_dataset`: The BigQuery dataset to train on.
- `bq_orders_table` : The BigQuery table of raw data.
- `bq_training_table` : The BigQuery table of training data after preprocessing.
- `bq_train_configuration` : The job configuration of training component
- `bq_model_table`: The BigQuery table of trained model.
- `bq_evaluate_time_series_configuration` : The job configuration of the evaluation of ARIMA time series
- `bq_evaluate_model_configuration` : The job configuration of the evaluation of ARIMA model
- `performance_threshold` : The value of average MAE threshold 
- `bq_forecast_configuration` : The job configuration of forecast component
- `bq_explain_forecast_configuration` : The job configuration of the evaluation of forecast component
- `project`: The project ID
- `location`: The location


In [ ]:
bqml_pipeline = vertex_ai.PipelineJob(
    display_name=f"{PIPELINE_NAME}-job",
    template_path=PIPELINE_PACKAGE,
    pipeline_root=PIPELINE_ROOT,
    enable_caching=False,
)

bqml_pipeline.run()

### View BigQuery ML training pipeline results

Finally, you will view the artifact outputs of each task in the pipeline.

In [ ]:
PROJECT_NUMBER = bqml_pipeline.gca_resource.name.split("/")[1]
print("PROJECT NUMBER: ", PROJECT_NUMBER)
print("\n\n")


def print_pipeline_output(job, output_task_name):
    JOB_ID = job.name
    print(JOB_ID)
    for _ in range(len(job.gca_resource.job_detail.task_details)):
        TASK_ID = job.gca_resource.job_detail.task_details[_].task_id
        EXECUTE_OUTPUT = (
            PIPELINE_ROOT
            + "/"
            + PROJECT_NUMBER
            + "/"
            + JOB_ID
            + "/"
            + output_task_name
            + "_"
            + str(TASK_ID)
            + "/executor_output.json"
        )
        GCP_RESOURCES = (
            PIPELINE_ROOT
            + "/"
            + PROJECT_NUMBER
            + "/"
            + JOB_ID
            + "/"
            + output_task_name
            + "_"
            + str(TASK_ID)
            + "/gcp_resources"
        )
        EVAL_METRICS = (
            PIPELINE_ROOT
            + "/"
            + PROJECT_NUMBER
            + "/"
            + JOB_ID
            + "/"
            + output_task_name
            + "_"
            + str(TASK_ID)
            + "/evaluation_metrics"
        )
        if tf.io.gfile.exists(EXECUTE_OUTPUT):
            ! gsutil cat $EXECUTE_OUTPUT
            return EXECUTE_OUTPUT
        elif tf.io.gfile.exists(GCP_RESOURCES):
            ! gsutil cat $GCP_RESOURCES
            return GCP_RESOURCES
        elif tf.io.gfile.exists(EVAL_METRICS):
            ! gsutil cat $EVAL_METRICS
            return EVAL_METRICS

    return None


print("bigquery-create-model-job")
artifacts = print_pipeline_output(bqml_pipeline, "bigquery-create-model-job")
print("\n\n")
print("bigquery-ml-arima-evaluate-job")
artifacts = print_pipeline_output(bqml_pipeline, "bigquery-ml-arima-evaluate-job")
print("\n\n")
print("get-model-evaluation-metrics")
artifacts = print_pipeline_output(bqml_pipeline, "get-model-evaluation-metrics")
print("\n\n")
print("bigquery-forecast-model-job")
artifacts = print_pipeline_output(bqml_pipeline, "bigquery-forecast-model-job")
print("\n\n")
print("bigquery-explain-forecast-model-job")
artifacts = print_pipeline_output(bqml_pipeline, "bigquery-explain-forecast-model-job")
print("\n\n")

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial.

In [ ]:
# delete pipeline
vertex_ai_pipeline_jobs = vertex_ai.PipelineJob.list(
    filter=f'pipeline_name="{PIPELINE_NAME}"'
)
for pipeline_job in vertex_ai_pipeline_jobs:
    pipeline_job.delete()

# delete model
DELETE_MODEL_SQL = f"DROP MODEL {BQ_DATASET}.{BQ_MODEL_TABLE}"
try:
    delete_model_query_job = bq_client.query(DELETE_MODEL_SQL)
    delete_model_query_result = delete_model_query_job.result()
except Exception as e:
    print(e)

# delete dataset
try:
    delete_detaset_query_result = bq_client.delete_dataset(
        BQ_DATASET, delete_contents=True, not_found_ok=True
    )
    print(delete_detaset_query_result)
except Exception as e:
    print(e)

# delete bucket
delete_bucket = True
if os.getenv("IS_TESTING") or delete_bucket:
    ! gsutil -m rm -r $BUCKET_URI


# Remove local resorces
! rm -rf {KFP_COMPONENTS_PATH}
! rm -rf {PIPELINES_PATH}